This block just imports the potential_fitting module.

In [1]:
import potential_fitting

SyntaxError: invalid syntax (potential_fitting.py, line 5)

Now define filepaths as variables so we don't have to keep retyping strings.
Make sure you have nothing of value in any of these files, as all files will be overwritten by certain steps of the workflow except settings, unopt_geo, and database.

In [4]:
settings = "CO2monomer.ini"
unopt_geo = "CO2.xyz"
opt_geo = "CO2.opt.xyz"
normal_modes = "normal_modes.dat"
configs = "config_files.xyz"
database = "CO2database.db"
training_set = "training_set.xyz"
poly_in = "A1B2.in"
molecule_in = "A1B2"
poly_directory = "poly"
config = "config.ini"
fit_directory = "fit"
fitted_code = "fitted"

This call creates the log directory as specified in settings.ini:files:log_path.
        In the future this will not be necessary, as the code will create any directories before using them.

In [5]:
potential_fitting.create_dirs(settings)

Optimizes the single geometry specified in the file located at the path specifed by unopt_geo and writes the optimized geometry to the file path specified by opt geo. This may take a while depending on what method/basis you use.

In [6]:
potential_fitting.optimize_geometry(settings, unopt_geo, opt_geo)

NameError: name 'Fragment' is not defined

Does a frequency calculation on the optimized geometry in the file located at the opt_geo path in order to generate a normal modes input file for the normal distribution configuration generator in the file located at normal_modes. dim_null (3 times number of atoms minus number of normal modes) is returned and should be used as input for the next function call. This may take a while depending on what method/basis you use.

In [5]:
dim_null = potential_fitting.generate_normal_modes(settings, opt_geo, normal_modes)

Beginning normal modes calculation using psi4 of CO2 with HF/STO-3G/
Normal mode/frequency analysis complete. 4 normal modes found


Uses the optimized geometry specified by opt_geo and the normal modes data file specified by normal_modes to generate a number of 1b configurations in the file specified by configs. dim_null is the return value of the above step.

In [6]:
potential_fitting.generate_1b_configurations(settings, opt_geo, normal_modes, dim_null, configs)

Running normal distribution configuration generator...
Normal Distribution Configuration generation complete.


This puts the configurations located in the file specified by configs into the database specified by database. It will initialize the database if it does not already exist, if it exists, new configurations will be added from the configs file. Model, basis, and cp to be used to calculate energy are also set at this time as specifed in the settings file. Note that cp (counterpoise correction) has no effect for 1b systems. The second call adds the optimized geometry to the database.

In [7]:
potential_fitting.init_database(settings, database, configs)
potential_fitting.init_database(settings, database, opt_geo)

Initializing database from xyz files in config_files.xyz directory into database ./CO2database.db
Initializing of database ./CO2database.db successful
Initializing database from xyz files in CO2.opt.xyz directory into database ./CO2database.db
Initializing of database ./CO2database.db successful


Loops through every uncalculated energy in the database and calculates it. This will take a while depending on what method/basis you use.

In [8]:
potential_fitting.fill_database(settings, database)

Filling database CO2database.db
     1     2     3     4     5     6     7     8     9    10
    11    12    13    14    15    16    17
Filling of database CO2database.db successful


Generates a training set in the file specified by training_set from all "CO2" (molecules with the name "CO2") energies in the database, additional arguments can be added to prune by method, basis, cp, or tag. If your database contains energies computed with a variety of methods/bases, it is probably a good idea to use only one of them in your training set at a time. The training set contains each configuration's deformation energy. The optimized geometry will have energy 0.

In [9]:
potential_fitting.generate_1b_training_set(settings, database, training_set, "CO2")

Creating a fitting input file from database CO2database.db into file ./training_set.xyz


This call generates a polynomial input file based on the symmetry of molecule "CO2" in the database, it will be generated in the current working directory

In [10]:
potential_fitting.generate_poly_input_from_database(settings, database, "CO2", ".")

Generate polynomials of degree 3 based on the polynomial input file as specified by poly_in (should be the file created by the above call, will be symmetry as specified in settings file + ".in" for 1b) in the directory specified by poly_directory

In [11]:
potential_fitting.generate_polynomials(settings, poly_in, 3, poly_directory)

Convert the polynomial maple files in poly_directory into cpp files

In [12]:
potential_fitting.execute_maple(settings, poly_directory)

Does a qchem calculation to generate the config file containing charges, polarizabilities, etc in the file specified by config based on the molecule symmetry specified by molecule_in (same as polynomial input but without the .in) and the optimized geometry as specified by opt_geo. This will take a while, but should not be effected by your method/basis.

In [13]:
potential_fitting.generate_fit_config(settings, molecule_in, config, opt_geo)

Generated the 1b fit code of degree 3 in fit_directory using the config file specified by config, and the polynomial files in poly_directory

In [14]:
potential_fitting.generate_1b_fit_code(settings, config, poly_in, poly_directory, 3, fit_directory)

Executing python generator script


Compile the fit code in fit_directory

In [15]:
potential_fitting.compile_fit_code(settings, fit_directory)

Generate a fitted .nc file at location "fit.nc" using the fit code in fit_directory, the training set as specified by training_set, and the fitcode as specified by fit_directory + "/fit-1b".

In [16]:
potential_fitting.fit_1b_training_set(settings, fit_directory + "/fit-1b", training_set, fit_directory, "fit.nc")